In [48]:
## import statements

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [5]:
#Loading the data

loader=PyPDFLoader(r"C:\Users\harsu\OneDrive\Anesthesia Texts\Guideline PDFs\2006 CMAJ Neurologic Determination of Death.pdf")
docs=loader.load()

In [36]:
# Making Chunks

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
chunks=text_splitter.split_documents(docs)

### Creating Embeddings

In [37]:
# Load Hugging Face token into environment for API access

os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [38]:
# Create text embeddings using Hugging Face's MiniLM model

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### Creating Vector Database

In [39]:
vectordb=Chroma.from_documents(documents=chunks,embedding=embeddings)

In [40]:
# Saving to the disk

vectordb=Chroma.from_documents(documents=chunks,embedding=embeddings,persist_directory="./data/chroma_db")

In [41]:
# Loading from the disk

db= Chroma(persist_directory="./data/chroma_db", embedding_function=embeddings )

### Setting Up LLM Access

In [42]:
# Set Groq API key from environment variables 

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [43]:
# Load Groq Llama Model

llm=ChatGroq(model="llama-3.1-8b-instant")
llm.invoke("Hey I am Harrie and i am Msc, applied computing student")

AIMessage(content="Hello Harrie, nice to meet you. MSc in Applied Computing, that's a great field of study. What aspect of Applied Computing are you most interested in? Artificial Intelligence, Data Science, Cyber Security, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 49, 'total_tokens': 97, 'completion_time': 0.053120887, 'prompt_time': 0.002841218, 'queue_time': 0.188086557, 'total_time': 0.055962105}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_3ddc9808b3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e2f0cd96-71a0-4d11-9027-c1c760066ff2-0', usage_metadata={'input_tokens': 49, 'output_tokens': 48, 'total_tokens': 97})

In [44]:
# Chatprompt Template

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
question: {input}
<context>
{context}
</context>

"""
)

In [47]:
# Document Chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [50]:
# Retriever Chain

retriever=db.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [51]:
# Getting the response form the LLM

response=retrieval_chain.invoke({"input":"What should be the minimum body temperature for clinical criteria"})
response['answer']

'The minimum body temperature for clinical criteria should be ≥ 34°C.'